In [1]:
import os
import pandas as pd
import numpy as np

result_dir = './results/'
data_dir = '../Documents/CantarCSV/'
figure_dir = "./figures/"
os.listdir(data_dir)



['14head.csv',
 '18head.csv',
 '2014 data.csv',
 '2015 data.csv',
 '2016 data.csv',
 '2017 data.csv',
 'Carrefour_2019_2020_2021_prepared2_02_11_2023.dta',
 'Datos_2018.csv',
 'Datos_2019.csv',
 'Datos_2020.csv',
 'Datos_2021.csv',
 'Datos_2022.csv',
 'Explicacion_VariablesV2.xlsx',
 'Explicacion_VariablesV3.xlsx']

In [4]:
read_able_columns = pd.read_excel(data_dir+'Explicacion_VariablesV3.xlsx', engine='openpyxl')


In [5]:

#codevf vs coproduct??
product_attributes=['CodProducto','CodigoBarras','CodVF', 'VF', 'SA7_BPL', 'SA4_Fabricante', 'SA3_Marca',       'SA2_Submarca',
       'X102_Aditivos', 'X108_Usuario', 'X109_Estado', 'X112_Niv_Calorias',
       'X124_Estilo', 'X127_Tipo_Bañado_Choc', 'X129_Tipo_Pasteleria',
       'X135_Uso', 'X143_Tipo_Corte', 'X163_Info_Cafeina',
       'X167_Grado_Curacion', 'X169_Cont_Materia_Grasa', 'X174_Despiece.Corte',
       'X182_Tipo_Queso', 'X188_Tipo_Pan', 'X189_Tipo_Viena',
       'X206_Tipo_Pescado', 'X230_Niv_Concentracion',
       'X314_Tipo_Cena.Entrante', 'X315_Tipo_Carne', 'X323_Tipo_Sazonamiento',
       'X328_Tipo_Aceite', 'X329_Pureness_Niveau', 'X360_Tipo_Verdura',
       'X366_Tipo_Postre', 'X383_Tipo_Yogurt', 'X384_Tipo_Cena',
       'X489_Como_Compro.', 'X490_Fresca.Congelada', 'X491_Presentacion',
       'X497_Tipo_Queso', 'X498_Marcas_Queso', 'X500_Tipo_Embutido',
       'X514_Tipo_Carne', 'X527_Tipo_Cafe', 'X529_Tipo_Producto',
       'X531_Info_Biologica', 'X558_Variedad_Queso', 'X571_Peso_Bebe',
       'X674_Tipo_Helado', 'X901_Localizacion', 'X902_Tipo_Producto',
       'X903_Envase', 'X904_Variedad.Sabor', 'X5_Num_Tot_Unidades',
       'X18_Num_Tot_Paquetes.Botes', 'X66_Preferred_Unit_Measure',]

purchase_attributes=['idCompra', 'CodPanelista','CodigoBarras', 'CodProducto','FechaCesta',	'PrecioCompra', 'Cantidad', 'Precio', 'CodLugarCompra','CodIndividuo','TipoMaquina', 'Promo_Folleto', 'Promo_Envase', 'Promo_TPR','promo']

shop_attributes=['CodLugarCompra','Lugar_Compra','Canal','CodIndividuo']


customer_attributes=['CodPanelista', 'CC_AA', 'Edad_Ama', 'NF', 'Numero_Gatos','Numero_Perros', 'Presencia_Niños', 'CodIMC', 'IMC', 'Ponderacion','Ciclo_Vida', 'Clase_EGM', 'Clase_Social', 'Habitat_Metropolitano',
       'Habitat_Municipal_Std', 'Inmigrante', 'Provincia', 'Region',]

tables={'customers': customer_attributes,'products': product_attributes,'shops':shop_attributes,'purchases':purchase_attributes }

In [6]:
# define mapping of types
type_dict = dict(zip(read_able_columns['Variable'].str.strip(), read_able_columns['Tipo Variable'].str.strip()))


type_translation = {'ID': 'int64', 'Fecha': 'datetime', 'Numérica': 'float64', 'Categórica': 'category',
                    'Dicotómica': 'bool'}
type_dict_new = {k: type_translation[v] for k, v in type_dict.items()}
type_dict_new = {('X' + k if k[0].isdigit() else k): v for k, v in type_dict_new.items()}
type_dict_new['CodigoBarras'] = 'str'
# TODO categorical vs int vs string...
# TODO check why it is needed , leave as categorical now..
type_dict_new['Presencia_Niños'] = 'category'

# when reading, should be given... 
del type_dict_new['FechaCesta']
# because stata cannot handle too many categories with long names:
for cn in product_attributes:
    if type_dict_new[cn] == 'category':
        type_dict_new[cn] = 'str'

In [17]:
def general_model(df):
    if df.shape[0]>30:
            # for throwing out outliers iteratively if we like, to have a better regression  
            cleaning_rounds=1
            Y = np.array(df['price_per_unit'])
            X = np.array(df['t'])
            #flag=np.zeros(y.size)
            idx=df[['price_per_unit']].index.copy()
            x=X.copy()
            y=Y.copy()
            print(x)
            for i in range(cleaning_rounds):
                p = np.polyfit(x, y, deg=5)
                ps = np.polyval(p, x)
                diff = abs(y-ps)
                std=np.std(diff)
                mean=np.mean(diff)
                good =  abs(diff-mean)/std < 3 
                #good = abs(y - ps) < 1  # Here we will only remove positive outliers
                    
                x_bad, y_bad = x[~good], y[~good]
                x, y = x[good], y[good]
        
           
        
                if (~good).sum() == 0:
                    break
            ps = np.polyval(p, X)
            diff = abs(Y-ps)
            std=np.std(diff)
            mean=np.mean(diff)
            z= abs(diff-mean)/std
            good =  z < 3 
            flag = np.logical_not(good)
          
            ret= pd.DataFrame(index=idx, data=list(zip(ps, z,flag)), columns=['1','2','3'])
    else:
        Y = np.array(df['price_per_unit'])
        #flag=np.zeros(y.size)
        idx=df[['price_per_unit']].index.copy()
        y=Y.copy()
    
        std=np.std(Y)
        mean=np.mean(Y)
        ps = np.full(y.shape, mean)
    
        z= abs(Y-mean)/std
        good =  z < 3 
        flag = np.logical_not(good)
      
        ret= pd.DataFrame(index=idx, data=list(zip(ps, z,flag)), columns=['1','2','3'])


    return ret


def get_oultiers(odf1):
       odf1['t'] = odf1['FechaCesta'].astype('datetime64').dt.dayofyear
       # calculate price per unit of sales
       odf1['price_per_unit']= odf1['Precio']/odf1['Cantidad']
       x=  odf1.groupby(['CodProducto'])[['t', 'price_per_unit']].apply(general_model)
       x1 = x.reset_index().set_index('level_1').drop(['CodProducto'], axis=1)
       odf1[['predicted', 'z', 'outlier']] = x1
       return odf1

In [50]:
import traceback
from sqlalchemy import create_engine
import codecs


all_products=None
all_customers=None
all_shops=None

input_enc="ISO-8859-1"#'latin_1 '
#input_enc= 'utf-16-be'

def fix_enc(fn,nrows):

    source_df_test=pd.read_csv(data_dir+fn,
                                encoding=input_enc,
                                delimiter="," ,on_bad_lines='skip',nrows=nrows)
    replace_chars={"Ã.": "ñ", "Ã³":"ó","Ã":"í"}  
    orig_names=source_df_test.columns.to_list()
    #re_encoded_names = [s.encode(input_enc) for s in orig_names]
    re_encoded_names = [codecs.decode(w.encode(input_enc)) for w in orig_names]

    #re_encoded_names = [s.decode("") for s in re_encoded_names]

    column_rename = dict(zip(source_df_test.columns.to_list()
                             , re_encoded_names))
    for k in source_df_test.columns:
        #print(k," -- ",source_df_test[k].dtype )
        if source_df_test[k].dtype == "str":
            try:
                source_df_test[k]=source_df_test[k].str.encode(input_enc)
                source_df_test[k]=source_df_test[k].str.decode('utf-8')
            except Exception as e:
                pass
                # print("EXC -- ",e,k)
        

    #source_df_test= source_df_test.replace(replace_chars)
    column_names=source_df_test.columns
    #for k,v in replace_chars.items():
    #    column_names=[cn.replace(k,v) for cn in column_names]
    column_rename = dict(zip(source_df_test.columns.to_list()
                             , column_names))
    source_df_test = source_df_test.rename(columns=column_rename)
    return source_df_test

def tell_me_about(s): return (type(s), r'{}'.format(s))

def load_and_preprocess_data(year, nrows=None):
    # load data
    
    if year<2018: 
        #decimal=","
        #enc="utf-8"
        input_fn = f'{year} data.csv'
        #delimiter=","
        source_df_test=fix_enc(input_fn,nrows=nrows,)
    else: 
        input_fn=f'Datos_{year}.csv'
        enc=input_enc 
        delimiter=";"
        decimal="."
        source_df_test=pd.read_csv(data_dir+input_fn,nrows=nrows,
                                     encoding = enc, 
                                    delimiter=delimiter ,decimal=decimal,on_bad_lines='skip', parse_dates=['FechaCesta'])
    print(source_df_test.columns)

    # transform 
    #if nrows:
    #    print(source_df_test.iloc[0][['idCompra','Promo_Envase','Promo_Folleto','Promo_TPR']])
    """
    source_df_test['promo_envase'] = source_df_test['promo_envase'].map({'No': False, 'Si': True})
    source_df_test['promo_folleto'] = source_df_test['promo_folleto'].map({'No': False, 'Si': True})
    source_df_test['Promo_TPR'] = source_df_test['Promo_TPR'].map({'No': False, 'Si': True})
    """
    #if nrows:

    #    print(source_df_test.iloc[0][['idCompra','Promo_Envase','Promo_Folleto','Promo_TPR']])

    '''
    for k in type_dict_new.keys():
       if k not in source_df_test.columns:
           print(source_df_test.columns)
           print(k)
    '''
    #source_df_test = source_df_test.astype(type_dict_new)
    for k,v in type_dict_new.items():
            try:
                # if v!="str":
                    #source_df_test[k] = source_df_test[k].astype('str')#.str.encode("utf-8")
                    source_df_test[k] = source_df_test[k].astype(v)
                    '''
                    if k == "CC_AA":
                        print('CC_AA --- ',v)
                        print('str --- ',source_df_test[k].iloc[:3])
                        w = source_df_test[k].iloc[1]
                       
                        print(w,' -> ',codecs.decode(w.encode("ISO-8859-1"),'utf-8'))
                    '''
                

                #else:
                #    source_df_test[k]=source_df_test[k].astype('str').str.encode("utf-8")
                #    if k == "CC_AA":
                #        print('str --- ',source_df_test[k].iloc[:3])
            except Exception as e:
                print("EXC -- ",repr(e),k,v)
                traceback.print_exc()
                
                #print(source_df_test[k].head())
    
       
    source_df_test['promo'] = np.where(
           source_df_test['Promo_Folleto'] | source_df_test['Promo_Envase'] | source_df_test['Promo_TPR'], True, False)
    return source_df_test


def separate_and_save(df, dfs,engine):
   # separating the tables:
    for name, attributes in tables.items():
  
        data = df[attributes]
        if name == 'purchases':
            to_table(name,data,engine)
            del df
            #data = get_oultiers(data)     
        else:
            data = data.drop_duplicates()
            dfs[name].append(data)
    
        #fn = result_dir+f'{year}_{name}.dta'
        #dfs[name].append(fn)
        
           
           
        #to_table(name,data)
        #all_data = pd.concat(df_list, ignore_index=True)
   
    return dfs


import json 
from sqlalchemy_utils import drop_database, database_exists,create_database
f=open("dbconnection.json")
connection_data = json.load(f)
DB_USER =connection_data["DB_USER"]

DB_PASS =connection_data["DB_PASS"]
DB_HOST =connection_data["DB_HOST"]
DB_PORT =connection_data["DB_PORT"]
DATABASE =connection_data["DATABASE"]
CHARSET ="latin_1"

   

connect_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}?charset={}'.format(DB_USER, DB_PASS, DB_HOST, DB_PORT, DATABASE,CHARSET)
connect_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(DB_USER, DB_PASS, DB_HOST, DB_PORT, DATABASE)


if database_exists(connect_string):
    drop_database(connect_string)
create_database(connect_string)

engine = create_engine(connect_string, connect_args={'client_encoding':CHARSET})

def to_table(name, df,engine,mode='append'):
    from sqlalchemy import create_engine
    df.to_sql(name, engine, if_exists=mode,method = 'multi', index=False)

#print(type_dict_new)


In [51]:
test = True
dfs={name:[] for name in tables.keys()}
nrows=None
if test:
    nrows=10
 
# first separate and save 
for year in range(2014,2023):
    print(year)
    df= load_and_preprocess_data(year,nrows)
    #df=get_oultiers(df)
    

    dfs = separate_and_save(df,dfs, engine)
    
    
    for name, l in dfs.items():
        if name=='purchases':
            continue
        data = pd.concat(l, ignore_index=True).drop_duplicates()
        to_table(name,data,engine,"append")  

2014


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 12: invalid continuation byte

In [63]:
df1=pd.read_csv("test_data/"+'14head.csv', encoding="iso-8859-1")


In [66]:
problem_word2=df1.columns[18]
problem_word2

'Presencia_NiÃ\x83.os'

In [67]:
problem_word1=df1['CC_AA'].iloc[0]
problem_word1

'AragÃ\x83Â³n'

'AragÃ³n'

In [149]:
pd.read_csv("test_data/"+'18head.csv')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [75]:
import codecs
codeclist = ["IBM850","IBM858",'ibm1140',"windows-1252",'iso-8859-1','iso-8859-15','macroman','utf_32','utf_32_be','utf_32_le','utf_16','utf_16_be','utf_16_le','utf_7','utf_8','utf_8_le','utf_8_be','utf_8_sig','iso-8859-16']
for codec in codeclist:
    try: 
        print(codec,": ",codecs.decode(problem_word2.encode("ISO-8859-1"),codec)  )
    except Exception as e: print(e)

'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' codec can't encode character '\u0102' in position 12: ordinal not in range(256)
'latin-1' 

In [69]:
for enc_c in codeclist:
    for dec_c in codeclist:
        try: 
            res=codecs.decode(problem_word2.encode(enc_c),dec_c  )
            if "ñ" in res:
                print(enc_c,'->',dec_c, ": ",res)
        except Exception as e:
            pass          

ibm1140 -> macroman :  ◊ôÖ¢ÖïÉâÅm’âf#Kñ¢


In [76]:
for enc_c in codeclist:
    try: 

        df1 = pd.read_csv("test_data/" + '14head.csv', encoding=enc_c,delimiter=",")
        print(enc_c," -> ",problem_word2)
    except Exception as e: print(enc_c,e)

IBM850  ->  Presencia_NiĂ.os
IBM858  ->  Presencia_NiĂ.os
ibm1140  ->  Presencia_NiĂ.os
windows-1252  ->  Presencia_NiĂ.os
iso-8859-1  ->  Presencia_NiĂ.os
iso-8859-15  ->  Presencia_NiĂ.os
macroman  ->  Presencia_NiĂ.os
utf_32 'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)
utf_32_be 'utf-32-be' codec can't decode bytes in position 0-3: code point not in range(0x110000)
utf_32_le 'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)
utf_16 UTF-16 stream does not start with BOM
utf_16_be  ->  Presencia_NiĂ.os
utf_16_le  ->  Presencia_NiĂ.os
utf_7 'utf7' codec can't decode byte 0xc3 in position 229: unexpected special character
utf_8  ->  Presencia_NiĂ.os
utf_8_le unknown encoding: utf_8_le
utf_8_be unknown encoding: utf_8_be
utf_8_sig  ->  Presencia_NiĂ.os
iso-8859-16  ->  Presencia_NiĂ.os


In [11]:
dfs

{'customers': ['./results/2014_customers.dta',
  './results/2015_customers.dta',
  './results/2016_customers.dta',
  './results/2017_customers.dta',
  './results/2018_customers.dta',
  './results/2019_customers.dta',
  './results/2020_customers.dta',
  './results/2021_customers.dta',
  './results/2022_customers.dta'],
 'products': ['./results/2014_products.dta',
  './results/2015_products.dta',
  './results/2016_products.dta',
  './results/2017_products.dta',
  './results/2018_products.dta',
  './results/2019_products.dta',
  './results/2020_products.dta',
  './results/2021_products.dta',
  './results/2022_products.dta'],
 'shops': ['./results/2014_shops.dta',
  './results/2015_shops.dta',
  './results/2016_shops.dta',
  './results/2017_shops.dta',
  './results/2018_shops.dta',
  './results/2019_shops.dta',
  './results/2020_shops.dta',
  './results/2021_shops.dta',
  './results/2022_shops.dta'],
 'purchases': ['./results/2014_purchases.dta',
  './results/2015_purchases.dta',
  './resu

In [21]:
year = 2014

input_fn=f'Datos_{year}.csv'
enc="ISO-8859-1"
delimiter=";"
decimal="."
if year<2018: 
    decimal=","
    enc="utf-8"
    input_fn = f'{year} data.csv'
    delimiter=","
   
source_df_test=pd.read_csv(data_dir+input_fn,
                          encoding = enc, delimiter=delimiter ,decimal=decimal,error_bad_lines=False, parse_dates=['FechaCesta'])

/Users/peterkiss/anaconda3/envs/kantar_data_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,66,68) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
ids_o = source_df_test['idCompra']
duplicated_purchase_ids_orig = source_df_test[ids_o.isin(ids_o[ids_o.duplicated()])]
duplicated_purchase_ids_orig.shape[0]


9654

In [ ]:
duplicated_purchase_ids.shape[0]

In [5]:
dfs={name:[] for name in tables.keys()}
for name in dfs.keys():
    for year in range(2014,2023):
        dfs[name].append(result_dir+f'{year}_{name}_u.dta')


In [14]:
dfs

{'customers': ['./results/2014_customers_u.dta',
  './results/2015_customers_u.dta',
  './results/2016_customers_u.dta',
  './results/2017_customers_u.dta',
  './results/2018_customers_u.dta',
  './results/2019_customers_u.dta',
  './results/2020_customers_u.dta',
  './results/2021_customers_u.dta',
  './results/2022_customers_u.dta'],
 'products': ['./results/2014_products_u.dta',
  './results/2015_products_u.dta',
  './results/2016_products_u.dta',
  './results/2017_products_u.dta',
  './results/2018_products_u.dta',
  './results/2019_products_u.dta',
  './results/2020_products_u.dta',
  './results/2021_products_u.dta',
  './results/2022_products_u.dta'],
 'shops': ['./results/2014_shops_u.dta',
  './results/2015_shops_u.dta',
  './results/2016_shops_u.dta',
  './results/2017_shops_u.dta',
  './results/2018_shops_u.dta',
  './results/2019_shops_u.dta',
  './results/2020_shops_u.dta',
  './results/2021_shops_u.dta',
  './results/2022_shops_u.dta'],
 'purchases': ['./results/2014_purch

In [ ]:
from pandas.api.types import union_categoricals





def load_yearly_data(name,dfs):
    return [pd.read_stata(fn) for fn in dfs[name]] 
    
for name in dfs.keys():
    df_list=load_yearly_data(name,dfs )
    # todo not sure if it is necessary
    print(f'unifying categoricals in {name} tables...')
    for c in df_list[0].columns:
        if df_list[0][c].dtype.name=='category':
            #print('cat union', c)
            #for df_per_year in dfs_per_year:
            #    print(c,y, df_per_year[c].cat.categories.tolist())
            uc = union_categoricals([df_per_year[c]  for df_per_year in df_list], ignore_order=True)
            for df_per_year in df_list:
                df_per_year[c]=pd.Categorical( df_per_year[c], categories=uc.categories )
    y=2014
    for df_y in  df_list:
        save_dta(result_dir+f'{y}_{name}_u.dta',df_y.drop_duplicates())   
        y+=1
    all_data = pd.concat(df_list)
    del df_list
    if name == 'purchases':
        duplicate_id_is=all_data['idCompra'].duplicated().any()
        print("duplicate purchases: ",duplicate_id_is)
     
    all_data = all_data.drop_duplicates()
    
    '''
    if 'NF' in all_data.columns:
        print(all_data['NF'])
        print(all_data['NF'].astype('category').dtype.name)

    '''
    save_dta(result_dir+f'all_{name}.dta',all_data)
    del all_data

'''
https://stackoverflow.com/questions/45639350/retaining-categorical-dtype-upon-dataframe-concatenation
uc = union_categoricals([df1.x,df2.x])
df1.x = pd.Categorical( df1.x, categories=uc.categories )
df2.x = pd.Categorical( df2.x, categories=uc.categories )
'''       


unifying categoricals in customers tables...
unifying categoricals in products tables...
unifying categoricals in shops tables...
unifying categoricals in purchases tables...
duplicate purchases:  True


In [7]:

#

 

In [27]:
df_list[0].drop('level_0',axis=1)

,level_0,index,idCompra,CodPanelista,CodigoBarras,CodProducto,FechaCesta,PrecioCompra,Cantidad,Precio,...,TipoMaquina,Promo_Folleto,Promo_Envase,Promo_TPR,promo,t,price_per_unit,predicted,z,outlier
0,0,0,48348413,187220,3123930711000,11649,2013-12-30,2233.0,2000.0,258.0,...,1,1,1,1,1,364,0.129000,0.118486,0.098759,0
1,1,1,48348415,187220,8410066001026,46899,2013-12-30,2233.0,3000.0,186.0,...,1,1,1,1,1,364,0.062000,0.065639,0.423578,0
2,2,2,48348416,187220,8410069021649,58452,2013-12-30,2233.0,1000.0,94.0,...,1,1,1,1,1,364,0.094000,0.090003,0.017334,0
3,3,3,48348417,187220,KIA1020201,74597,2013-12-30,2233.0,3000.0,169.0,...,1,1,1,1,1,364,0.056333,0.048225,0.495195,0
4,4,4,48348418,187220,8421136700015,313733,2013-12-30,2233.0,1000.0,165.0,...,1,1,1,1,1,364,0.165000,0.199167,0.786573,0


In [11]:
def deduplicate(dslst,fnlst):
    res=[]
    for n,d in zip(fnlst,dslst):
        print('dropping duplictes ',n)
        d=d.drop_duplicates("idCompra", ignore_index=True)
        d=d.drop('level_0',axis=1)
        print('saving deduplicated ',n)
        save_dta(n,d)
        print('done ',n)

        res.append(d)
    return res

df_list=deduplicate(df_list,dfs[name])


dropping duplictes  ./results/2014_purchases_u.dta
saving deduplicated  ./results/2014_purchases_u.dta
done  ./results/2014_purchases_u.dta
dropping duplictes  ./results/2015_purchases_u.dta
saving deduplicated  ./results/2015_purchases_u.dta
done  ./results/2015_purchases_u.dta
dropping duplictes  ./results/2016_purchases_u.dta
saving deduplicated  ./results/2016_purchases_u.dta
done  ./results/2016_purchases_u.dta
dropping duplictes  ./results/2017_purchases_u.dta
saving deduplicated  ./results/2017_purchases_u.dta
done  ./results/2017_purchases_u.dta
dropping duplictes  ./results/2018_purchases_u.dta
saving deduplicated  ./results/2018_purchases_u.dta
done  ./results/2018_purchases_u.dta
dropping duplictes  ./results/2019_purchases_u.dta
saving deduplicated  ./results/2019_purchases_u.dta
done  ./results/2019_purchases_u.dta
dropping duplictes  ./results/2020_purchases_u.dta
saving deduplicated  ./results/2020_purchases_u.dta
done  ./results/2020_purchases_u.dta
dropping duplictes  

In [2]:
df_list[0].head()

NameError: name 'df_list' is not defined

concat done


In [ ]:

def load_yearly_data(name,dfs):
    return [pd.read_stata(fn) for fn in dfs[name]] 



name ="purchases"
df_list=load_yearly_data(name,dfs )
#all_data = pd.concat(df_list, ignore_index=True)
#del df_list
#save_dta(result_dir+f'all_{name}.dta',all_data.drop_duplicates())
#del all_data
all_data = pd.concat(df_list, ignore_index=True)
print('concat done')
if 'level_0' in all_data.columns:
    all_data= all_data.drop('level_0',axis=1)
    print('dropp done')
else: print('no level_0')
print("entries ",all_data.shape[0])
save_dta(result_dir+f'all_{name}.dta',all_data)
print('saving done')


concat done
dropp done
entries  67198136


In [1]:
df_list[0].sort_values("idCompra").head()

NameError: name 'df_list' is not defined

In [10]:
ids=all_data['idCompra']
duplicated_purchase_ids=all_data[ids.isin(ids[ids.duplicated()])]


In [17]:
d= df_list[0]
ids=d['idCompra']
duplicated_purchase_ids=d[ids.isin(ids[ids.duplicated()])]

In [18]:
duplicated_purchase_ids.shape[0]

9654

In [12]:
duplicated_purchase_ids.head()

,level_0,index,idCompra,CodPanelista,CodigoBarras,CodProducto,FechaCesta,PrecioCompra,Cantidad,Precio,...,TipoMaquina,Promo_Folleto,Promo_Envase,Promo_TPR,promo,t,price_per_unit,predicted,z,outlier
914,914,914,48349862,180894,8410436221054,923524,2013-12-30,3500.0,2000.0,630.0,...,6,1,1,1,1,364,0.315,0.312130,0.634186,0
915,915,915,48349862,180894,8410436221054,923524,2013-12-30,3500.0,2000.0,630.0,...,6,1,1,1,1,364,0.315,0.312130,0.634186,0
5001,5001,5001,48356043,217885,8480017752635,492555,2013-12-30,1931.0,2000.0,600.0,...,5,1,1,1,1,364,0.300,0.287322,0.364919,0
5002,5002,5002,48356043,217885,8480017752635,492555,2013-12-30,1931.0,2000.0,600.0,...,5,1,1,1,1,364,0.300,0.287322,0.364919,0
5592,5592,5592,48356941,174212,8480017752635,492555,2013-12-30,650.0,2000.0,600.0,...,6,1,1,1,1,364,0.300,0.287322,0.364919,0


In [ ]:
duplicated_purchase_ids.to_csv(result_dir+'duplicated_purchase_ids.csv')


In [13]:
del duplicated_purchase_ids

In [9]:
all_data.head()

,level_0,index,idCompra,CodPanelista,CodigoBarras,CodProducto,FechaCesta,PrecioCompra,Cantidad,Precio,...,TipoMaquina,Promo_Folleto,Promo_Envase,Promo_TPR,promo,t,price_per_unit,predicted,z,outlier
0,0,0,48348413,187220,3123930711000,11649,2013-12-30,2233.0,2000.0,258.0,...,1,1,1,1,1,364,0.129000,0.118486,0.098759,0
1,1,1,48348415,187220,8410066001026,46899,2013-12-30,2233.0,3000.0,186.0,...,1,1,1,1,1,364,0.062000,0.065639,0.423578,0
2,2,2,48348416,187220,8410069021649,58452,2013-12-30,2233.0,1000.0,94.0,...,1,1,1,1,1,364,0.094000,0.090003,0.017334,0
3,3,3,48348417,187220,KIA1020201,74597,2013-12-30,2233.0,3000.0,169.0,...,1,1,1,1,1,364,0.056333,0.048225,0.495195,0
4,4,4,48348418,187220,8421136700015,313733,2013-12-30,2233.0,1000.0,165.0,...,1,1,1,1,1,364,0.165000,0.199167,0.786573,0


In [ ]:
all_data=all_data.drop_duplicates()

In [7]:
all_data=all_data.drop('level_0',axis=1)

In [ ]:
save_dta(result_dir+f'all_{name}.dta',all_data.drop_duplicates())

In [3]:
all_data.head()

NameError: name 'all_data' is not defined

In [9]:
duplicated_purchase_ids=all_data[ids.isin(ids[ids.duplicated()])]


In [1]:
print(x)

NameError: name 'x' is not defined

In [10]:
duplicated_purchase_ids.to_csv(result_dir+'duplicated_purchase_ids.csv')

In [11]:
duplicated_purchase_ids

,index,idCompra,CodPanelista,CodigoBarras,CodProducto,FechaCesta,PrecioCompra,Cantidad,Precio,CodLugarCompra,...,TipoMaquina,Promo_Folleto,Promo_Envase,Promo_TPR,promo,t,price_per_unit,predicted,z,outlier
914,914,48349862,180894,8410436221054,923524,2013-12-30,3500.0,2000.0,630.0,164,...,6,1,1,1,1,364,0.315,0.312130,0.634186,0
915,915,48349862,180894,8410436221054,923524,2013-12-30,3500.0,2000.0,630.0,164,...,6,1,1,1,1,364,0.315,0.312130,0.634186,0
5001,5001,48356043,217885,8480017752635,492555,2013-12-30,1931.0,2000.0,600.0,33,...,5,1,1,1,1,364,0.300,0.287322,0.364919,0
5002,5002,48356043,217885,8480017752635,492555,2013-12-30,1931.0,2000.0,600.0,33,...,5,1,1,1,1,364,0.300,0.287322,0.364919,0
5592,5592,48356941,174212,8480017752635,492555,2013-12-30,650.0,2000.0,600.0,33,...,6,1,1,1,1,364,0.300,0.287322,0.364919,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8286790,8286790,167801087,443752,20047696,951733,2020-01-20,2654.0,1000.0,155.0,68,...,Desconocida,1,1,1,1,20,0.155,0.153950,0.098650,0
8286823,8286823,167836411,443752,20078270,769493,2020-01-20,2654.0,1000.0,119.0,68,...,Desconocida,1,1,1,1,20,0.119,0.143701,0.603377,0
8286832,8286832,167803107,443752,20798093,333441,2020-01-20,2654.0,1000.0,179.0,68,...,Desconocida,1,1,1,1,20,0.179,0.182424,1.057499,0
8286896,8286896,168186578,443752,3560071085889,698133,2020-05-04,3511.0,2000.0,460.0,252,...,Desconocida,1,1,1,1,125,0.230,0.216955,1.768609,0


In [52]:
year=2014
input_fn=f'Datos_{year}.csv'
if year<2018: 
    input_fn = f'{year} data.csv'

def fix_enc(fn):

    source_df_test=pd.read_csv(fn,
                              encoding='ISO-8859-1', delimiter="," ,=False)
    replace_chars={"Ã.": "ñ", "Ã³":"ó","Ã":"í"}  
    source_df_test= source_df_test.replace(replace_chars)
    column_names=source_df_test.columns
    for k,v in replace_chars.items():
        column_names=[cn.replace(k,v) for cn in column_names]
        source_df_test=source_df_test.replace(k,v)
    column_rename = dict(zip(source_df_test.columns.to_list()
                             , column_names))

    source_df_test = source_df_test.rename(columns=column_rename)
    source_df_test.to_csv(fn)
for year in range(2014,2018):
    print(year)
    input_fn = f'{year} data.csv'

    fix_enc(data_dir+ input_fn)

2014


TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [105]:
problem_word=source_df_test.columns[18]

In [106]:
problem_word

'Presencia_Niños'

In [67]:
import codecs
codeclist = ["IBM850","IBM858",'ibm1140',"windows-1252",'iso-8859-1','iso-8859-15','macroman','utf_32','utf_32_be','utf_32_le','utf_16','utf_16_be','utf_16_le','utf_7','utf_8','utf_8_sig','iso-8859-16']
for codec in codeclist:
    try: 
        print(codec,": ",codecs.decode(problem_word.encode("ISO-8859-1"),codec)  )
    except Exception as e: print(e)


IBM850 :  Presencia_Ni├.os
IBM858 :  Presencia_Ni├.os
ibm1140 :  &ÊÁËÁ>ÄÑ/¬+ÑC?Ë
windows-1252 :  Presencia_NiÃ.os
iso-8859-1 :  Presencia_NiÃ.os
iso-8859-15 :  Presencia_NiÃ.os
macroman :  Presencia_Ni√.os
'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)
'utf-32-be' codec can't decode bytes in position 0-3: code point not in range(0x110000)
'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)
utf_16 :  牐獥湥楣彡楎⻃獯
utf_16_be :  偲敳敮捩慟乩쌮潳
utf_16_le :  牐獥湥楣彡楎⻃獯
'utf7' codec can't decode byte 0xc3 in position 12: unexpected special character
'utf-8' codec can't decode byte 0xc3 in position 12: invalid continuation byte
'utf-8' codec can't decode byte 0xc3 in position 12: invalid continuation byte
iso-8859-16 :  Presencia_NiĂ.os


In [74]:
w= 'Bañado'
for enc_c in codeclist:
    for dec_c in codeclist:
        try: 
            print(enc_c,'->',dec_c, ": ",codecs.decode(w.encode(enc_c),dec_c  ))
        except Exception as e: print(e)

codecs.decode(sp_n,codec)

IBM850 -> IBM850 :  Bañado
IBM850 -> IBM858 :  Bañado
IBM850 -> ibm1140 :  â/u/À?
IBM850 -> windows-1252 :  Ba¤ado
IBM850 -> iso-8859-1 :  Ba¤ado
IBM850 -> iso-8859-15 :  Ba€ado
IBM850 -> macroman :  Ba§ado
'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)
'utf-32-be' codec can't decode bytes in position 0-3: code point not in range(0x110000)
'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)
IBM850 -> utf_16 :  慂憤潤
IBM850 -> utf_16_be :  䉡ꑡ摯
IBM850 -> utf_16_le :  慂憤潤
'utf7' codec can't decode byte 0xa4 in position 2: unexpected special character
'utf-8' codec can't decode byte 0xa4 in position 2: invalid start byte
'utf-8' codec can't decode byte 0xa4 in position 2: invalid start byte
IBM850 -> iso-8859-16 :  Ba€ado
IBM858 -> IBM850 :  Bañado
IBM858 -> IBM858 :  Bañado
IBM858 -> ibm1140 :  â/u/À?
IBM858 -> windows-1252 :  Ba¤ado
IBM858 -> iso-8859-1 :  Ba¤ado
IBM858 -> iso-8859-15 :  Ba€ado
IBM858 -> macroman 

'Ă±'

In [130]:
years=[2014,2022]
for year in years:
    input_fn=f'Datos_{year}.csv'
    enc="ISO-8859-1"
    sep=";"
    decimal="."
    if year<2018: 
        decimal=","
        enc="utf-8"
        input_fn = f'{year} data.csv'
        sep=","
    df=pd.read_csv(data_dir+input_fn,nrows=100,encoding=enc,delimiter=sep,decimal=decimal)
    print(df.columns)

    print(df['Ponderacion'].head())


Index(['Unnamed: 0', 'Unnamed: 0.1', 'idCompra', 'CodPanelista', 'CodProducto',
       'CodigoBarras', 'FechaCesta', 'PrecioCompra', 'Cantidad', 'Precio',
       'CodLugarCompra', 'Lugar_Compra', 'Canal', 'CodIndividuo', 'CC_AA',
       'Edad_Ama', 'NF', 'Numero_Gatos', 'Numero_Perros', 'Presencia_Niños',
       'CodIMC', 'IMC', 'Ponderacion', 'CodVF', 'VF', 'SA7_BPL',
       'SA4_Fabricante', 'SA3_Marca', 'SA2_Submarca', 'X102_Aditivos',
       'X108_Usuario', 'X109_Estado', 'X112_Niv_Calorias', 'X124_Estilo',
       'X127_Tipo_Bañado_Choc', 'X129_Tipo_Pasteleria', 'X135_Uso',
       'X143_Tipo_Corte', 'X163_Info_Cafeina', 'X167_Grado_Curacion',
       'X169_Cont_Materia_Grasa', 'X174_Despiece.Corte', 'X182_Tipo_Queso',
       'X188_Tipo_Pan', 'X189_Tipo_Viena', 'X206_Tipo_Pescado',
       'X230_Niv_Concentracion', 'X314_Tipo_Cena.Entrante', 'X315_Tipo_Carne',
       'X323_Tipo_Sazonamiento', 'X328_Tipo_Aceite', 'X329_Pureness_Niveau',
       'X360_Tipo_Verdura', 'X366_Tipo_Postre', '

In [169]:
type_dict_new

{'idCompra': 'int64',
 'CodProducto': 'int64',
 'CodigoBarras': 'str',
 'PrecioCompra': 'float64',
 'Cantidad': 'float64',
 'Precio': 'float64',
 'CodLugarCompra': 'int64',
 'Lugar_Compra': 'category',
 'Canal': 'category',
 'CodPanelista': 'int64',
 'CodIndividuo': 'category',
 'CC_AA': 'category',
 'Ciclo_Vida': 'category',
 'Clase_EGM': 'category',
 'Clase_Social': 'category',
 'Edad_Ama': 'float64',
 'Habitat_Metropolitano': 'category',
 'Habitat_Municipal_Std': 'category',
 'Inmigrante': 'bool',
 'NF': 'category',
 'Numero_Gatos': 'category',
 'Numero_Perros': 'float64',
 'Presencia_Niños': 'category',
 'Provincia': 'category',
 'Region': 'category',
 'CodIMC': 'category',
 'IMC': 'category',
 'Ponderacion': 'float64',
 'CodVF': 'str',
 'VF': 'str',
 'SA7_BPL': 'bool',
 'SA4_Fabricante': 'str',
 'SA3_Marca': 'str',
 'SA2_Submarca': 'str',
 'X102_Aditivos': 'str',
 'X108_Usuario': 'str',
 'X109_Estado': 'str',
 'X112_Niv_Calorias': 'str',
 'X124_Estilo': 'str',
 'X127_Tipo_Bañado_C

In [4]:
name='purchases'
year='2014'

p1=pd.read_stata(result_dir+f'{year}_{name}.dta',encoding="iso-8859-1")
name='products'

p2=pd.read_stata(result_dir+f'{year}_{name}.dta')


In [5]:
p1.head()

,index,idCompra,CodPanelista,CodigoBarras,CodProducto,FechaCesta,PrecioCompra,Cantidad,Precio,CodLugarCompra,CodIndividuo,TipoMaquina,Promo_Folleto,Promo_Envase,Promo_TPR,promo,outlier,z
0,0,48348413,187220,3123930711000,11649,2013-12-30,2233.0,2000.0,258.0,120,0,1,1,1,1,1,0,0.098759
1,1,48348415,187220,8410066001026,46899,2013-12-30,2233.0,3000.0,186.0,120,0,1,1,1,1,1,0,0.423578
2,2,48348416,187220,8410069021649,58452,2013-12-30,2233.0,1000.0,94.0,120,0,1,1,1,1,1,0,0.017334
3,3,48348417,187220,KIA1020201,74597,2013-12-30,2233.0,3000.0,169.0,120,0,1,1,1,1,1,0,0.495195
4,4,48348418,187220,8421136700015,313733,2013-12-30,2233.0,1000.0,165.0,120,0,1,1,1,1,1,0,0.786573


In [6]:
p2.head()

,index,CodProducto,CodigoBarras,CodVF,VF,SA7_BPL,SA4_Fabricante,SA3_Marca,SA2_Submarca,X102_Aditivos,...,X558_Variedad_Queso,X571_Peso_Bebe,X674_Tipo_Helado,X901_Localizacion,X902_Tipo_Producto,X903_Envase,X904_Variedad_Sabor,X5_Num_Tot_Unidades,X18_Num_Tot_Paquetes_Botes,X66_Preferred_Unit_Measure
0,0,11649,3123930711000,207,QUESO RALLADO,1,143160,30388,30388,nan,...,nan,nan,nan,FRESCOS,QUESO RALLADO,BOLSA PLASTICO,VALOR INDISPONIBLE,1.0,NaN,100.0
1,1,46899,8410066001026,145,TOMATE NATURAL EN CONSERVA,1,32651,173012,173012,nan,...,nan,nan,nan,ESTANTERIA,TOMATE,LATA,NATURAL,1.0,NaN,400.0
2,2,58452,8410069021649,204,HARIN/SEMOL/TAPIOC/ALMDN/QUINO,1,160970,10863,10863,nan,...,nan,nan,nan,ESTANTERIA,HARINA,PAQUETE,EXTRA,1.0,NaN,1000.0
3,3,74597,KIA1020201,462,Pan fresco (CB),1,122091,122091,122091,nan,...,nan,nan,nan,nan,nan,nan,nan,1.0,NaN,210.0
4,4,313733,8421136700015,11,FRUTA Y VERDURA FRESCA,1,282084,1759,1759,nan,...,nan,nan,nan,ESTANTERIA,SETA,BANDEJA,nan,1.0,NaN,200.0


In [21]:
name='customers'

p3=pd.read_stata(result_dir+f'{year}_{name}.dta')

In [30]:
p3.head()


,index,CodPanelista,CC_AA,Edad_Ama,NF,Numero_Gatos,Numero_Perros,Presencia_Ni_os,CodIMC,IMC,...,Ciclo_Vida,Clase_EGM,Clase_Social,Habitat_Metropolitano,Habitat_Municipal_Std,Inmigrante,Provincia,Region,CC_AA1,CC_AA2
0,0,187220,AragÃ³n,60.0,4,0,1.0,No,3.0,Sobrepeso,...,Parejas con hijos edad media,M,Alta y media Alta,NO METROP,10001 a 20000,1,HUESCA,Region I,b'Arag\xc3\xb3n',Aragón
1,21,167538,Castilla y LeÃ³n,68.0,2,0,0.0,No,2.0,Normal,...,Retirados,M,Media baja,NO METROP,50001 a 100000,1,ZAMORA,Region IV,b'Castilla y Le\xc3\xb3n',Castilla y León
2,35,174704,AndalucÃ­a,45.0,3,2,0.0,NiÃ±os 6-15,3.0,Sobrepeso,...,Parejas con hijos edad media,MB,Media baja,NO METROP,500001 a 1000000,1,MALAGA,Region III,b'Andaluc\xc3\xada',Andalucía
3,42,174566,CataluÃ±a,47.0,4,1,0.0,NiÃ±os 6-15,4.0,Obesidad,...,Hogares Monoparentales,A,Media,NO METROP,5001 a 10000,1,BARCELONA,Region I,b'Catalu\xc3\xb1a',Cataluña
4,107,210543,Comunitat Valenciana,41.0,4,1,1.0,NiÃ±os 6-15,3.0,Sobrepeso,...,Parejas con hijos edad media,M,Media,AM VALENCIA,20001 a 30000,1,VALENCIA,Region II,b'Comunitat Valenciana',Comunitat Valenciana


In [28]:
p3['CC_AA1']=p3['CC_AA'].str.encode(encoding="iso-8859-1")


In [29]:
p3['CC_AA2']=p3['CC_AA1'].str.decode(encoding="utf-8")


In [8]:
name='shops'
p4=pd.read_stata(result_dir+f'{year}_{name}.dta')
p4.head()

,index,CodLugarCompra,Lugar_Compra,Canal,CodIndividuo
0,0,120,SUPERMERCADO,super+autos,0
1,9,123,TIENDAS,tda.de alimentacion,0
2,17,124,DIRECTAS,resto canales,0
3,19,127,CARNICERIA,carniceria/polleria,0
4,21,134,PANADERIA,panaderia/horno,1


# Testing 

In [13]:


year=2014
input_fn=f'Datos_{year}.csv'
enc="ISO-8859-1"
delimiter=";"
decimal="."
if year<2018: 
    decimal=","
    enc="utf-8"
    input_fn = f'{year} data.csv'
    delimiter=","
# read 3 lines from the yearly source file
df_orig=pd.read_csv(data_dir+input_fn,nrows=3,encoding="iso-8859-1",delimiter=delimiter ,decimal=decimal,
                          error_bad_lines=False, parse_dates=['FechaCesta'])


In [16]:
names=df_orig.columns.tolist()

In [18]:
def remove_sp_char(text):
    res= ''.join([i if ord(i) < 128 else '_' for i in text])
    return res.replace('__','_')

fixed_names =[remove_sp_char(n) for n in names]
fixed_names

['Unnamed: 0',
 'Unnamed: 0.1',
 'idCompra',
 'CodPanelista',
 'CodProducto',
 'CodigoBarras',
 'FechaCesta',
 'PrecioCompra',
 'Cantidad',
 'Precio',
 'CodLugarCompra',
 'Lugar_Compra',
 'Canal',
 'CodIndividuo',
 'CC_AA',
 'Edad_Ama',
 'NF',
 'Numero_Gatos',
 'Numero_Perros',
 'Presencia_Ni_os',
 'CodIMC',
 'IMC',
 'Ponderacion',
 'CodVF',
 'VF',
 'SA7_BPL',
 'SA4_Fabricante',
 'SA3_Marca',
 'SA2_Submarca',
 'X102_Aditivos',
 'X108_Usuario',
 'X109_Estado',
 'X112_Niv_Calorias',
 'X124_Estilo',
 'X127_Tipo_Ba_ado_Choc',
 'X129_Tipo_Pasteleria',
 'X135_Uso',
 'X143_Tipo_Corte',
 'X163_Info_Cafeina',
 'X167_Grado_Curacion',
 'X169_Cont_Materia_Grasa',
 'X174_Despiece.Corte',
 'X182_Tipo_Queso',
 'X188_Tipo_Pan',
 'X189_Tipo_Viena',
 'X206_Tipo_Pescado',
 'X230_Niv_Concentracion',
 'X314_Tipo_Cena.Entrante',
 'X315_Tipo_Carne',
 'X323_Tipo_Sazonamiento',
 'X328_Tipo_Aceite',
 'X329_Pureness_Niveau',
 'X360_Tipo_Verdura',
 'X366_Tipo_Postre',
 'X383_Tipo_Yogurt',
 'X384_Tipo_Cena',
 'X48

In [19]:
name_map = dict(zip(names, fixed_names))

In [80]:
lines=[]

#df_full=df_per_year=pd.read_stata(result_dir+f'{year}_purchases.dta')
all_c=pd.read_stata(result_dir+f'all_customers.dta')
all_c = all_c.loc[:, ~all_c.columns.str.contains('^Unnamed')]

all_s=pd.read_stata(result_dir+f'all_shops.dta')
all_s = all_s.loc[:, ~all_s.columns.str.contains('^Unnamed')]

all_pr=pd.read_stata(result_dir+f'all_products.dta')
all_pr = all_pr.loc[:, ~all_pr.columns.str.contains('^Unnamed')]

#all_pr=df_orig.rename(columns=name_map)
#all_s=df_orig.rename(columns=name_map)

#names = df_orig.columns.tolist()


def remove_sp_char(text):
    res = ''.join([i if ord(i) < 128 else '_' for i in text])
    return res.replace('__', '_')



for year in range(2014,2015):
    # load data
    input_fn=f'Datos_{year}.csv'
    enc="ISO-8859-1"
    delimiter=";"
    decimal="."
    if year<2018: 
        decimal=","
        enc="utf-8"
        input_fn = f'{year} data.csv'
        delimiter=","
    # read 3 lines from the yearly source file
    '''df_orig=pd.read_csv(data_dir+input_fn,nrows=3,encoding="iso-8859-1",delimiter=delimiter ,decimal=decimal,
                          error_bad_lines=False, parse_dates=['FechaCesta'])'''
    df_orig=load_and_preprocess_data(year, 3)
    '''
    replace_chars={"Ã.": "ñ", "Ã³":"ó","Ã":"í"}  
    df_orig= df_orig.replace(replace_chars)
    column_names=df_orig.columns
    for k,v in replace_chars.items():
        column_names=[cn.replace(k,v) for cn in column_names]
    column_rename = dict(zip(df_orig.columns.to_list()
                             , column_names))

    df_orig = df_orig.rename(columns=column_rename)
    '''
    print(df_orig.iloc[0][['idCompra','Promo_Envase','Promo_Folleto','Promo_TPR']])
    df_orig.to_stata('orig.dta')
    df_orig=pd.read_stata('orig.dta')
    print(df_orig.iloc[0][['idCompra','Promo_Envase','Promo_Folleto','Promo_TPR']])


    names=df_orig.columns.tolist()

    fixed_names = [remove_sp_char(n) for n in names]
    name_map = dict(zip(names, fixed_names))

    df_orig=df_orig.rename(columns=name_map)

    #print(df_orig.columns)
    idCompras=df_orig['idCompra'].tolist()
    #load the yearly purchase  
    df_per_year_p=pd.read_stata(result_dir+f'{year}_purchases_u.dta')
    df_per_year_p = df_per_year_p.loc[:, ~df_per_year_p.columns.str.contains('^Unnamed')]

    df_per_year_c=pd.read_stata(result_dir+f'{year}_customers_u.dta')
    df_per_year_c = df_per_year_c.loc[:, ~df_per_year_c.columns.str.contains('^Unnamed')]

    df_per_year_s=pd.read_stata(result_dir+f'{year}_shops_u.dta')
    df_per_year_s = df_per_year_s.loc[:, ~df_per_year_s.columns.str.contains('^Unnamed')]

    df_per_year_pr=pd.read_stata(result_dir+f'{year}_products_u.dta')
    df_per_year_pr = df_per_year_pr.loc[:, ~df_per_year_pr.columns.str.contains('^Unnamed')]

    df_per_year_p_1=pd.read_stata(result_dir+f'{year}_purchases.dta')
    df_per_year_p_1 = df_per_year_p_1.loc[:, ~df_per_year_p_1.columns.str.contains('^Unnamed')]
    df_per_year_c_1=pd.read_stata(result_dir+f'{year}_customers.dta')
    df_per_year_c_1 = df_per_year_c_1.loc[:, ~df_per_year_c_1.columns.str.contains('^Unnamed')]

    df_per_year_s_1=pd.read_stata(result_dir+f'{year}_shops_u.dta')
    df_per_year_s_1 = df_per_year_s_1.loc[:, ~df_per_year_s_1.columns.str.contains('^Unnamed')]

    df_per_year_pr_1=pd.read_stata(result_dir+f'{year}_products_u.dta')
    df_per_year_pr_1 = df_per_year_pr_1.loc[:, ~df_per_year_pr_1.columns.str.contains('^Unnamed')]


    df_per_year_p=df_per_year_p[df_per_year_p['idCompra'].isin(idCompras)]
    df_per_year_p_1=df_per_year_p.copy()
    df_per_year_p=df_per_year_p.merge(df_per_year_c).merge(df_per_year_s).merge(df_per_year_pr)
    df_per_year_p_2=df_per_year_p_1.merge(df_per_year_c_1).merge(df_per_year_s_1).merge(df_per_year_pr_1)

    df_per_year_p_1=df_per_year_p.merge(all_c).merge(all_s).merge(all_pr)
    df_orig['origin']='orig'
    df_per_year_p['origin']='per_year'
    df_per_year_p_1['origin']='per_year_all'  
    df_per_year_p_2['origin']='per_year_all_sep_cat'  


    #df_per_year_p=pd.reed_stata(result_dir+f'{year}_purchase.dta')
    lines.append(df_orig)
    print('df_orig',df_orig.columns.tolist())
    lines.append(df_per_year_p)
    print('df_per_year_p',df_per_year_p.columns.tolist())

    lines.append(df_per_year_p_1)
    print('df_per_year_p_1',df_per_year_p_1.columns.tolist())

    lines.append(df_per_year_p_2)
    print('df_per_year_p_2',df_per_year_p_2.columns.tolist())


res=pd.concat(lines)


idCompra         48348413
Promo_Envase           No
Promo_Folleto          No
Promo_TPR              No
Name: 0, dtype: object
idCompra         48348413
Promo_Envase        False
Promo_Folleto       False
Promo_TPR           False
Name: 0, dtype: object
idCompra         48348413
Promo_Envase         True
Promo_Folleto        True
Promo_TPR            True
Name: 0, dtype: object


/Users/peterkiss/anaconda3/envs/kantar_data_env/lib/python3.7/site-packages/pandas/io/stata.py:2398: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Unnamed: 0   ->   Unnamed__0
    Unnamed: 0.1   ->   Unnamed__0_1
    Presencia_Niños   ->   Presencia_Ni_os
    X127_Tipo_Bañado_Choc   ->   X127_Tipo_Ba_ado_Choc
    X174_Despiece.Corte   ->   X174_Despiece_Corte
    X314_Tipo_Cena.Entrante   ->   X314_Tipo_Cena_Entrante
    X489_Como_Compro.   ->   X489_Como_Compro_
    X490_Fresca.Congelada   ->   X490_Fresca_Congelada
    X904_Variedad.Sabor   ->   X904_Variedad_Sabor
    X18_Num_Tot_Paquetes.Botes   ->   X18_Num_Tot_Paquetes_Botes

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)


idCompra         48348413
Promo_Envase            1
Promo_Folleto           1
Promo_TPR               1
Name: 0, dtype: object
df_orig ['index', 'Unnamed_0', 'Unnamed_0_1', 'idCompra', 'CodPanelista', 'CodProducto', 'CodigoBarras', 'FechaCesta', 'PrecioCompra', 'Cantidad', 'Precio', 'CodLugarCompra', 'Lugar_Compra', 'Canal', 'CodIndividuo', 'CC_AA', 'Edad_Ama', 'NF', 'Numero_Gatos', 'Numero_Perros', 'Presencia_Ni_os', 'CodIMC', 'IMC', 'Ponderacion', 'CodVF', 'VF', 'SA7_BPL', 'SA4_Fabricante', 'SA3_Marca', 'SA2_Submarca', 'X102_Aditivos', 'X108_Usuario', 'X109_Estado', 'X112_Niv_Calorias', 'X124_Estilo', 'X127_Tipo_Ba_ado_Choc', 'X129_Tipo_Pasteleria', 'X135_Uso', 'X143_Tipo_Corte', 'X163_Info_Cafeina', 'X167_Grado_Curacion', 'X169_Cont_Materia_Grasa', 'X174_Despiece_Corte', 'X182_Tipo_Queso', 'X188_Tipo_Pan', 'X189_Tipo_Viena', 'X206_Tipo_Pescado', 'X230_Niv_Concentracion', 'X314_Tipo_Cena_Entrante', 'X315_Tipo_Carne', 'X323_Tipo_Sazonamiento', 'X328_Tipo_Aceite', 'X329_Pureness_Niveau

In [78]:
res = res.sort_values('idCompra')

In [79]:
res.to_csv('compare.csv')

In [74]:
res.columns.tolist()

['index',
 'Unnamed_0',
 'Unnamed_0_1',
 'idCompra',
 'CodPanelista',
 'CodProducto',
 'CodigoBarras',
 'FechaCesta',
 'PrecioCompra',
 'Cantidad',
 'Precio',
 'CodLugarCompra',
 'Lugar_Compra',
 'Canal',
 'CodIndividuo',
 'CC_AA',
 'Edad_Ama',
 'NF',
 'Numero_Gatos',
 'Numero_Perros',
 'Presencia_Ni_os',
 'CodIMC',
 'IMC',
 'Ponderacion',
 'CodVF',
 'VF',
 'SA7_BPL',
 'SA4_Fabricante',
 'SA3_Marca',
 'SA2_Submarca',
 'X102_Aditivos',
 'X108_Usuario',
 'X109_Estado',
 'X112_Niv_Calorias',
 'X124_Estilo',
 'X127_Tipo_Ba_ado_Choc',
 'X129_Tipo_Pasteleria',
 'X135_Uso',
 'X143_Tipo_Corte',
 'X163_Info_Cafeina',
 'X167_Grado_Curacion',
 'X169_Cont_Materia_Grasa',
 'X174_Despiece_Corte',
 'X182_Tipo_Queso',
 'X188_Tipo_Pan',
 'X189_Tipo_Viena',
 'X206_Tipo_Pescado',
 'X230_Niv_Concentracion',
 'X314_Tipo_Cena_Entrante',
 'X315_Tipo_Carne',
 'X323_Tipo_Sazonamiento',
 'X328_Tipo_Aceite',
 'X329_Pureness_Niveau',
 'X360_Tipo_Verdura',
 'X366_Tipo_Postre',
 'X383_Tipo_Yogurt',
 'X384_Tipo_Cena

In [48]:
all_pr.columns

Index(['index', 'CodProducto', 'CodigoBarras', 'CodVF', 'VF', 'SA7_BPL',
       'SA4_Fabricante', 'SA3_Marca', 'SA2_Submarca', 'X102_Aditivos',
       'X108_Usuario', 'X109_Estado', 'X112_Niv_Calorias', 'X124_Estilo',
       'X127_Tipo_Ba_ado_Choc', 'X129_Tipo_Pasteleria', 'X135_Uso',
       'X143_Tipo_Corte', 'X163_Info_Cafeina', 'X167_Grado_Curacion',
       'X169_Cont_Materia_Grasa', 'X174_Despiece_Corte', 'X182_Tipo_Queso',
       'X188_Tipo_Pan', 'X189_Tipo_Viena', 'X206_Tipo_Pescado',
       'X230_Niv_Concentracion', 'X314_Tipo_Cena_Entrante', 'X315_Tipo_Carne',
       'X323_Tipo_Sazonamiento', 'X328_Tipo_Aceite', 'X329_Pureness_Niveau',
       'X360_Tipo_Verdura', 'X366_Tipo_Postre', 'X383_Tipo_Yogurt',
       'X384_Tipo_Cena', 'X489_Como_Compro_', 'X490_Fresca_Congelada',
       'X491_Presentacion', 'X497_Tipo_Queso', 'X498_Marcas_Queso',
       'X500_Tipo_Embutido', 'X514_Tipo_Carne', 'X527_Tipo_Cafe',
       'X529_Tipo_Producto', 'X531_Info_Biologica', 'X558_Variedad_Queso',


In [49]:
len(df_orig.columns.tolist())

92

In [75]:
res.head()

,index,Unnamed_0,Unnamed_0_1,idCompra,CodPanelista,CodProducto,CodigoBarras,FechaCesta,PrecioCompra,Cantidad,...,promo_folleto,promo_envase,promo,origin,level_0,t,price_per_unit,predicted,z,outlier
0,0,0.0,1.0,48348413,187220,11649,3123930711000,2013-12-30,2233.0,2000.0,...,NaN,NaN,1,orig,NaN,NaN,NaN,NaN,NaN,NaN
0,0,NaN,NaN,48348413,187220,11649,3123930711000,2013-12-30,2233.0,2000.0,...,NaN,NaN,1,per_year,0.0,364.0,0.129,0.118486,0.098759,0.0
0,0,NaN,NaN,48348413,187220,11649,3123930711000,2013-12-30,2233.0,2000.0,...,NaN,NaN,1,per_year_all,0.0,364.0,0.129,0.118486,0.098759,0.0
0,0,NaN,NaN,48348413,187220,11649,3123930711000,2013-12-30,2233.0,2000.0,...,NaN,NaN,1,per_year_all_sep_cat,0.0,364.0,0.129,0.118486,0.098759,0.0
1,1,1.0,2.0,48348415,187220,46899,8410066001026,2013-12-30,2233.0,3000.0,...,NaN,NaN,1,orig,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
[c for c in res.columns.tolist() if c not in df_orig.columns.tolist()]

['level_0', 'promo', 't', 'price_per_unit', 'predicted', 'z', 'outlier']

In [48]:
[c for c in df_orig.columns.tolist() if c not in res.columns.tolist()]


[]

In [82]:
df_per_year_c.columns

Index(['level_0', 'index', 'CodPanelista', 'CC_AA', 'Edad_Ama', 'NF',
       'Numero_Gatos', 'Numero_Perros', 'Presencia_Ni_os', 'CodIMC', 'IMC',
       'Ponderacion', 'Ciclo_Vida', 'Clase_EGM', 'Clase_Social',
       'Habitat_Metropolitano', 'Habitat_Municipal_Std', 'Inmigrante',
       'Provincia', 'Region'],
      dtype='object')

In [83]:
df_per_year_c.reset_index()

ValueError: cannot insert level_0, already exists